# This Week’s Overview


In this practical we're going to expand our understanding of how to use `GeoPandas` as part of an analysis, and of how to implement different types of spatial and non-spatial joins in Python using the tools to-hand. You'll notice that the number of libraries that we need to do our work has been expanding since last week. You will have to finish 5 tasks:

- Load shapefile (using `geopandas`) and raster data (using `georasters` and `GDAL`).
- Non-spatial join - Some DataFrame operation such as 'join','append',’merge’ you learned in last term will be used.
- Spatial join - using python  - find points in polygons, convert raster to points and find points in polygons
- Overlaying raster and vector data type.
- Visualise spatial data using `geopandas` - revisit practical 8 in Geocomputation


## Learning Outcomes

By the end of this practical you should have:
- Enhanced your skills of data processing - spatial and non-spatial join
- A better understanding of raster and vector data type in practices

## Get prepared
(always remember) The first thing we need to do is setup our working environment. Run the scripts below to import the relevant libraries.

In [ ]:
import matplotlib as mpl
mpl.use('TkAgg')
# Make sure output is into notebook
%matplotlib inline
import matplotlib.pyplot as plt

import pysal as ps
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.cm as cm
import urllib
import zipfile
import re
import os

import shapely
from shapely.geometry import Point

import random 
random.seed(123456789) # For reproducibility

# Make numeric display a bit easier
pd.set_option('display.float_format', lambda x: '{:,.4f}'.format(x))

In [ ]:
import warnings
warnings.simplefilter('ignore')

Check if you have the package installed. If not, install the package from command line using codes below.
GeoRasters can be installed using pip or conda. Remember to activate gsa2018 - the kernal used for this practical. then, georaster package will be installed in gsa2018.

**pip install git+git://github.com/ozak/georasters.git**

**pip install georasters**

or

**conda install -c ozak georasters**

yml file is also available from here.

In [ ]:
import georasters as gr

## Downloading data

Let's start by (re)loading both the AirBnB and the LSOA data, together with the NS-SeC data from last term! This will give us the tools with which to test our understanding of data linking _and_ allow us to do some simple, but hopefully interesting, analyses. To try to reduce confusion, I've provided a way to automate the downloading of the data in the answers:
**Save the LDN-LSOAs and Airbnb listings data in your local directory, 

In [ ]:
# Create the data dir if it doesn't exist
if os.path.isdir('data') is not True:
    print("Creating 'data' directory...")
    os.mkdir('data')

# Configure the download
url  = 'http://www.reades.com/LDN-LSOAs.zip'
path = os.path.join("data","LDN-LSOAs.zip")

# Download
r    = urllib.request.urlretrieve(url, path)

# Unzip it into the data folder
z    = zipfile.ZipFile(path)
m    = z.extractall("data")

In [ ]:
# Configure the next download for NS-SeC data
url  = 'http://www.reades.com/NSSHRP_UNIT_URESPOP.zip'
path = os.???.join("data","NSSHRP_UNIT_URESPOP.zip")

# Download but don't unzip it
r    = urllib.request.urlretrieve(url, path)

In [ ]:
# Configure the next download -- notice 
# that you will need to visit InsideAirBnB
# in order to check that the data hasn't been
# updated from 2018/12/07.
url  = 'http://data.insideairbnb.com/united-kingdom/england/london/2018-12-07/data/listings.csv.gz'
path = os.path.join("data","listings.csv.gz")

# Download but don't unzip it
r    = ???.request.???(url, path)

The raster data you are going to play with is modelled air quality data available on London Datastore (modelled by King's Environmental research team!). Estimates of key pollutants (NOx, PM10, PM2.5 and CO2) are included for the base year 2013 and projected forward to 2020, 2025, and 2030. 

The LAEI (London Atmospheric Emissions Inventory) area covers the 32 London Boroughs and the City of London and up to the M25 motorway. You can find full data sets of air quality from [here](https://data.london.gov.uk/dataset/london-atmospheric-emissions-inventory-2013/resource/725450b0-a25c-4dc7-af30-c67054a676a3). 

To save you some time and also memory space of your computer, we have selected a small data set for your practice.

In [ ]:
# Configure the next download -- air quality data 
# we have prepared a smaller size raster data for you.
url  = 'https://github.com/daisy8738/sp/raw/master/Met2013_PM25.zip'
path = os.path.join("data","PM25.zip")

# Download the data
r    = urllib.request.urlretrieve(url, path)

# unzip it into the data folder
z    = zipfile.???(path)
m    = z.extractall("???")

**Ok, you have downloaded all data sets!** The next step is to load the data sets and get yourself familiar with the relevant packages. 

### Loading AirBnB Data

Here is all the code to load the AirBnB data into a GeoDataFrame called `sdf` (spatial sample  dataframe):

In [ ]:
# what format of data you are reading into the dataframe?
df       = pd.read_???(os.path.join('data','listings.csv.gz'))
sample   = df.sample(frac=0.1)
geometry = [Point(xy) for xy in zip(sample.longitude, sample.latitude)]
crs      = {'init': 'epsg:4326'} # What projection is this lat/long? please google it
sdf      = gpd.GeoDataFrame(sample, crs=crs, geometry=geometry)
sdf      = sdf.to_crs({'init': '???'}) # Reproject into OSGB1936 with epsg value 27700

# Check the output for specific variables
sdf.head(3)[['id','host_id','neighbourhood','price','geometry']]

**Now go to check the directory of your ipython-notebook file, a folder called 'data' is then there**

### Loading the LSOA Shapefile

Here is all of the code to load the LDN-LSOA shapefile, preparing the geopandas data frame for us to work with!

In [ ]:
lsoas = gpd.read_file(os.path.join('data','LDN-LSOAs.shp'))
lsoas.sample(3)

### Loading the NS-SeC Data

Let's pull together everything we learned last term about what data we need to keep from the NS-SeC data file with everything we learned _this_ term with how to load data directly from a Zip file without the need to unzip it! 

Also, when you get to the end of this code block take a look at the first three columns and see if there's anything that look like it might match what we have in the `lsoas` geo-data.

In [ ]:
z = zipfile.ZipFile(os.path.???('data','NSSHRP_UNIT_URESPOP.zip'))

nssec = pd.read_???(z.open('Data_NSSHRP_UNIT_URESPOP.csv'), skiprows=[1])

# If we find this column, this deletes it
if np.where(nssec.columns.values=='Unnamed: 15')[0]:
    del nssec['Unnamed: 15']
if np.where(nssec.columns.values=='GEO_TYPE')[0]:
    del nssec['GEO_TYPE']
if np.where(nssec.columns.values=='GEO_TYP2')[0]:
    del nssec['GEO_TYP2']

# Initialise column names
colnames = ['CDU','GeoCode','GeoLabel','Total']
# And append the remainder of groups to columns
for i in range(1,9):
    colnames.append('Group' + str(i))
colnames.append('NC')
nssec.columns = colnames

# Check the output
nssec.head(3)

### Loading the Air Quality (PM 2.5) data - have a go with raster file

GeoRasters package provides a fast and flexible tool to work with GIS raster files. It provides the GeoRaster class, which makes working with rasters quite transparent and easy. In a way it tries to do for raster what GeoPandas does for geometries.

We will first have a go with Georasters, as it allows a quick visualisation of raster data.

If very unfortunately, Package georasters does not work with you as it clashes with other dependencies, you can go to the very end of this practical, there gives another solution of using raster data by GDAL.

We are going to read .asc file. The ESRI ASCII raster format can be used to transfer information to or from other cell-based or raster systems. When an existing raster is output to an ESRI ASCII format raster, the file will begin with header information that defines the properties of the raster such as the cell size, the number of rows and columns, and the coordinates of the origin of the raster. The header information is followed by cell value information specified in space-delimited row-major order, with each row seperated by a carraige return.

In [ ]:
#import the airqality file just downloaded from your local folder
rasterpath = ???.path.join('data','2013_Met2013_LAEI2013_PM25.asc')

# Load raster data by calling georasters
rasterdata = gr.from_file(rasterpath)

In [ ]:
# Plot data
plt.rcParams["figure.figsize"] = (12,8)
rasterdata.plot(cmap = 'gist_earth')

In [ ]:
# what's the type of rasterdata
???(rasterdata)

In [ ]:
# Get some stats
print('average value of PM2.5 is %f' %rasterdata.mean())
print('min value of PM2.5 is %f' %rasterdata.???())
print('summed value of PM2.5 is %f' %rasterdata.sum())
print('standard deviation of PM2.5 is %f' %rasterdata.???())

#### Challenge 1 (Optional)
Could you find how many rows and columns in the raster file? 

**Hint**: now the rasterdata is like an array. how do you find the dimensions of an array?

In [ ]:
# your code here 
rasterdata.???

In [ ]:
#Get the coordinates in the raster file
(xmin, xsize, x, ymax, y, ysize) = rasterdata.geot

# can you calculate the boundary of aboved plotted raster file? 
# you already know xmin, then what is the xmax?
xmax = xmin + xsize*rasterdata.shape[1]
# get the value for xmax
???

In [ ]:
# you already know ymax, then what is the ymin?
ymin = ymax + ysize*rasterdata.shape[0]
# get the value for ymin
???

#### Challenge 2
You got the (xmin, ymin) and (xmax,ymax), could you transform the ($x,y$) in 2-d space to latitude and longtitude?

**Hint**: check practical 2 - in this case, you need to project **from epsg 27700 to epsg 4326**.

In [ ]:
from pyproj import Proj, transform

inProj = Proj(init='???')
outProj = Proj(init='???')
lon_min,lat_min = transform(inProj,outProj,xmin,ymin)
lon_max,lat_max = transform(inProj,outProj,xmax,ymax)
print (lon_min,lat_min,lon_max,lat_max)

Folium is actually quite a useful tool. we have used it in Practical 1 and 2, we can also overlap an image on map. Folium works with image file - tiff. So before we do mapping, lets convert the .asc file to tiff file (tiff is like an image), which can be easily done by Georasters.

In [ ]:
rasterdata.to_tiff('data/pm25_tiff')

In [ ]:
# call folium to realize the interactive map
import folium
from folium import plugins
from scipy.ndimage import imread

# boundary of the image on the map
min_lon = lon_min
max_lon = lon_max
min_lat = lat_min
max_lat = lat_max
# create the map
map = folium.???(location=[51.5113, -0.1160], tiles='Stamen Terrain', zoom_start = 10)

# read in png file to numpy array
data = imread('data/pm25_tiff.tif')

# Overlay the image
map.add_child(folium.raster_layers.ImageOverlay(data, cmap = 'gist_earth', opacity=0.8, bounds =[[min_lat, min_lon], [max_lat, max_lon]]))
map.save('PM25 map.html')
# or you can simply call 'map' for visualization without saving it

**Georasters can also convert raster file to pandas dataframe. **

In [ ]:
# Convert to Pandas DataFrame
raster_df = rasterdata.to_pandas()

In [ ]:
raster_df.head(3)

**What does x,y,value mean here? Is that easier to get boundary from raster_df?**

In [ ]:
print('xmin of rasterdata is %f' %raster_df.x.???())
print('xmax of rasterdata is %f' %raster_df.x.max())
print('ymin of rasterdata is %f' %raster_df.???.min())
print('ymax of rasterdata is %f' %raster_df.y.???())

print('the avergae value of PM2.5 in the measured area is %f' %raster_df.value.???())

## Joining Data by Key

This is the most common way to join data (spatial or not) together: we have a unique id in one dataset and we match it to the _same_ unique id in a different data set. You should have known several types of join very well, but lets have another review:

1. Inner join: this will keep only the rows that have matching unique ids in both data sets, so any unique id that doesn't match will be discarded.
2. Outer join: this will keep all rows in both data sets, regardless of whether or not there is a match in the other data set.
3. Left join: all rows in the 'left' data set will be kept, regardless of whether or not there is a match in the 'right' data set.
4. Right join: reverse of a left join.

Left and right joins are a matter of preference: there is no real difference between them, it just depends on which data set you think of as being 'first' (i.e. to the _left_ in the join).

**_Note:_** this may seem fairly straightforward so far ("A whole practical on four types of joins???"), but you are probably making several unconscious assumptions: 1) you are assuming that each key or location is unique; 2) that the data set is not so large that the structure of the index matters; 3) that you are not joining based on multiple simultaneous conditions (e.g. within this area _and_ of this type)... We're only going to scrape the surface here, but you should be aware the topic of how to store, manage, and link data is incredibly complex: in database design we talk of the importance of complying with the five normal forms to manage data integrity, and then we violate them to improve performance!

There is, unfortunately, one bit of ugliness that we have to grapple with and it ties back to ideas of inheritance: **`merge` is a _pandas_ function and pandas doesn't know about _geopandas_.** So if we naively use pandas' merge function on a spatial data set then we get back a pandas dataframe. If, instead, we use the geopandas dataframe then we'll get back a geopandas dataframe. Again, this is a question of judgement -- the pandas `merge` is a little easier to read and gives us some more flexiblity, but with more hoops to jump through aftwards... Use whichever makes the most sense to you.

#### Method 1: Pandas Merge

Although I've been unable to reproduce what happened while I was developing the practical, if you find that you are no longer working with a GeoPandas object after a merge or similar operation, then it is fairly easy to convert between Pandas and GeoPandas by simply passing the dataframe to the GeoPandas constructor along with the projection and geometry fields. That said, this seems to work nicely and is equivalent to Method 2 (you don't need to run both blocks):

In [ ]:
shpdf = pd.merge(lsoas, nssec, left_on='lsoa11cd', right_on='GeoCode', how='left')
print("Merged df is of type: " + str(type(shpdf)))

#### Method 2: Geopandas' Merge

This method is more direct, but it's not necessarily immediately clear what's going on because now `lsoas` is _implicitly_ the _left_ part of the join, not _explicitly_. Plus, if you were to reverse the order (so: `nssec.merge(lsoas...)`) then the result would no longer be a GeoDataFrame because you'd be using pandas' merge function, not geopandas'! Confusing, right?

In [ ]:
shpdf = lsoas.merge(nssec, left_on='lsoa11cd', right_on='GeoCode')
print("Shape df is of type: " + str(type(shpdf)))

You may find the types of output are identical from both methods! 

In [ ]:
print("lsoas has {0} rows, {1} columns".format(lsoas.shape[0], lsoas.shape[1]))
print("nssec has {0} rows, {1} columns".format(nssec.shape[0], nssec.shape[1]))
print("shpdf has {0} rows, {1} columns".format(shpdf.shape[0], shpdf.shape[1]))
shpdf.sample(3)

It's worth taking a few minutes to review what that one line of code just did and _why_ I chose to do it that way:

* First, note that we _could_ join on _either_ `LSOA11NM` and `GeoLabel` or `LSOA11CD` and `GeoCode` -- the result would have been the same in both cases, but as a _general rule_ you will usually get better results matching on something that has been _designed_ to be a key (as the 'Code' value) than on regular text (the 'Name' value). Think of this way: there are many different 'entities' that might be called 'London' in a data set (there's the City of London, the Greater London Authority, Inner London, etc.), whereas there is probably only going to be _one_ E010000001.
* Second, why did I _not_ use the `CDU` field when that _also_ appears to be a 'matchable' field? The answer to this is more subtle: it is _likely_ that the `CDU` is also some kind of unique identifier, but it's hard to tell from the documentation whether it's consistent across data sets from different sources: there are many uses for integer fields, and enabling you to link data is only one of them. In contrast, if you get an exact match on `E01001587` between two data sets then there is _no_ way that that has happened by chance alone. So the LSOA Code and Name fields are more 'robust' as linking fields than some integer.
* Third, why did I chose a left join when I just said that there's no real difference between left and right joins? In addition to what I wrote above about the potential confusion between pandas' merge function and geopandas' merge function, you also need to remember that `lsoas` contains the LSOAs that I can _map_, so there's not much point in keeping LSOAs that I can't actually map right? The NS-SeC data contains _every_ LSOA in England and Wales, so we don't really need those to be part of our analysis. If we had picked an _inner_ join instead then we'd have had a similar result _but_ we wouldn't have any way of knowing if we had lost data! If we have rows with _None_ values for any of the NS-SeC data then we know that we have a small problem thanks to the use of the left join.
* Finally, if this whole matter of joins is a bit confusing, I'd suggest playing with the _how_ parameter in the code block above to see how it changes the 'diagnostics' that I print out.

### Better Maps

We're now going to look at a slightly better way of creating maps -- we'll have less control over the output but we will at least get a legend this time! At this point we're heading towards the limits of what we can teach you without writing even longer notebooks. To really get to grips with how to create _good_ looking maps using code you'll have to familiarise yourself with `matplotlib` and you will also need to write your own legend function to customise placement and such. But it's both important _and_ useful to know that you could easily generate 20 maps _this_ way and they would be broadly acceptable for an IGS without even having to open up QGIS or Arc!

This approach uses a `scheme` parameter wrapped in plotting function by Geopandas. So now let's go ahead and calculate a Location Quotient for Group 1 _and_ map it using a variety of styles so that we can see how our understanding of the data changes with the different classification schemes -- we did this in Week 8 geocomputation! 

In [ ]:
shpdf['Group1Lq'] = (
    shpdf.Group1.values / shpdf.Total.astype(float).values) / (float(shpdf.Group1.sum()) / shpdf.Total.sum()
)

In [ ]:
shp_link = os.path.join('data','LDN-LSOAs-new.shp')
shp_link

In [ ]:
# save your shapefile to your folder
shpdf.to_file(shp_link)

In [ ]:
from IPython.core.pylabtools import figsize
# now we change the figure size to allow a bigger, nicer plot
figsize(14, 10)
# get it plotted
shpdf.plot()

Not nice enough though, Let's asign colors to LSOA zones by classification of Group1Lq, remember how to do it with geopandas?

In [ ]:
f, ax = plt.subplots(1, figsize=(14, 10))
base = shpdf.plot(ax=ax, column='Group1Lq', cmap='OrRd', scheme='equal_interval', edgecolor='white', linewidth=0.1, legend=True)

## Spatial Joins of Two Shapefiles

Simple spatial joins work much like the non-spatial ones we just looked at above, except that the link is made on the basis of location, not some identifier. This makes it much more likely that we'll have 1-to-Many joins, let me illustrate by _first_ using a non-spatial test. Let's have a look at the distribution of LSOAs by borough... but we firstly need to create a Borough variable from... which column contains Borough information indirectly?

In [ ]:
# You might want to investigate what I'm up to 
# with str.replace(...)
# think about the reason why we raise replace?
lsoas['Borough'] = lsoas.lsoa11nm.???.replace('\d\d\d\w$', '', case=False)
lsoas.groupby('Borough').size()

In other words, if we plan to do some work at, say, the Borough level then a join against the LSOA data will give us hundreds of matches! Remember that our AirBnB data is more granular than the LSOAs, so we can expect thousands of matches (even allowing for the fact that we have only taken a _subsample_ of the full data set). You will want to have a look at [the documentation](http://geopandas.org/mergingdata.html?highlight=sjoin#spatial-joins).

But before we complete the join, we need to check that the projections match!

In [ ]:
print("AirBnB CRS: " + str(sdf.crs))
print("LSOAs CRS: " + str(lsoas.crs))

Although a spatial join _should_ work with these since they are actually _essentially_ the same, **you should resolve this apparent mismatch** by reprojecting the LSOAs into the EPSG:27700 format (BNG) and _then_ join.

In [ ]:
lsoas = lsoas.to_crs({'init': '???'})

In [ ]:
airbnb = gpd.sjoin(sdf, lsoas, how="inner", op='within')
print("airbnb has {0} rows, {1} columns".format(airbnb.shape[0], airbnb.shape[1]))
airbnb.sample(3)[['id','name','neighbourhood','lsoa11nm','Borough']] # Remember Borough is from the LSOA Name

While that might look promising, there could be trouble lurking... it will help if we ensure that we're comparing like to like by removing any leading or trailing whitespace from neighbourhood and Borough names:

In [ ]:
# Tidy up the two strings so that they're more likely to match
airbnb['neighbourhood'] = airbnb.neighbourhood.str.strip()
airbnb['Borough'] = airbnb.Borough.str.strip()

# Are there any non-matching rows?
airbnb[(airbnb.neighbourhood.str.strip() != airbnb.Borough.str.strip()) & (airbnb.neighbourhood.notnull())][
    ['id','name','neighbourhood','Borough']]
airbnb.head(3)

My sample included a number of 'LB of'(London Borough of...) and 'RB of' (Royal Borough of...), which would also need to be stripped out in order to improve our matching of neighbourhoods:

In [ ]:
# Strip out 'LB of' and 'RB of'
airbnb['neighbourhood'] = airbnb.neighbourhood.str.replace('LB of ','').str.replace('RB of','')

# And repeat
airbnb[ (airbnb.neighbourhood.str.strip() != airbnb.Borough.str.strip()) & (airbnb.neighbourhood.notnull()) ].groupby(['Borough','neighbourhood']).size()

So... notice that there are some mismatches between neighbourhood and borough: in some cases this is because the neighbourhood value is at a higher resolution (_e.g._ Shoreditch is within Hackney) but it also looks like some properties are being 'upgraded' from one borough to another (_e.g._ Tower Hamlets to Hackney). In the full data set we might be able to evaluate the extent of this issue... 

**_Note:_** as an alternative to `dropna()`, we can all do `fillna()` when slicing through a data frame to extract rows using a column that contains NaNs.

In [ ]:
df.head(1)

In [ ]:
# Note: this is the full data set
kendf = df[df.name.str.contains('Kensington').fillna(False)]
print("kendf has {0} rows, {1} columns".format(kendf.shape[0], kendf.shape[1]))

# Fix the price data
kendf['price'] = kendf.loc[:,('price')].str.replace("$",'').str.replace(",",'').astype(float)

# Show what we've got
kendf.groupby('neighbourhood').size()

## Over To You: Testing Neighbourhood Listings

Well _that's_ interesting, let's explore! Pulling together the code you've used above (_and_ extending some of the ideas) try mapping out the AirBnB listings that sell themselves as being in Kensington and seeing if there is some kind of persistent bias. I should point out that, technically, they simply mention Kensington and so might be fairly honest in saying 'we are near Kensington...'

**Your map should focus on the area around Kensington, and not show all of London. **

**This will require to work out how to make use of the boundary/bounding box to define the range of view, or to create a subset of your data so that it's easy to clip to the range of the AirBnB listings...**

**_Remember_**: if you reall want to check that you've successfully written your own script then you should be able to _Restart_ the Kernel (from the drop-down) and then run _only_ the code block below in order to produce your analysis.

In [ ]:
# Create a geodata frame from the Kensington data frame
# your code here
# Hint: 4 lines of code including geometry, crs, sdf for kensington and reproject the sdf to OSGB1936.
???
???
???
???
print("Kensington listings geodata frame created...")

# Extract the Kensington LSOAs
borough_sdf = lsoas[lsoas.Borough.str.contains('Kensington')]
# borough_sdf.sample(3)
print("Kensington borough geodata frame created...")

In [ ]:
# Create the output dir if it doesn't exist and write the selected shapefile to the output folder
if os.path.isdir('output') is not True:
    print("Creating 'output' directory...")
    os.mkdir('output')
# save the shapefile you have created into "output" folder
borough_sdf.to_file(os.path.join('output','Kensington.shp'))
ken_sdf.???(os.path.join('output','KensingtonListings.shp'))
print("Shapefiles written.")

In [ ]:
the_borough = gpd.read_file(os.path.join('output','Kensington.shp'))
listings = gpd.read_file(os.path.join('output','KensingtonListings.shp'))

listings.sample(3)

In [ ]:
# count the length of listings
print("There are " + ???(len(listings)) + " rows in the listings.")

In [ ]:
prices = np.array(listings['price']) # Retrieve pricing data
pricesq5 = ps.esda.mapclassify.Quantiles(prices, k=5) # Classify into 5 quantiles
print(pricesq5) # Show the classification result

In [ ]:
figsize=(10,13) # Why do you think I changed the figure size this way?

base = the_borough.plot(color='white', edgecolor='black')

listings.plot(ax=base, marker='o', color='red', markersize=5);

The plotting seems not very nice, right? there is a point (outlier) far far away from the borough and stretches the view.


So how can we improve the view? We are going to get the bounding box of the borough, and then use the bounding box to defind the xlim, ylim of the view. In such case, the outlier point will not show in the view.


Geopandas has a full set of functions to get bounding box, buffered zones, outlines, read [here](http://geopandas.org/geometric_manipulations.html) to get more information. 

In [ ]:
bounds = borough_sdf.geometry.bounds # you get the bounding box of ken_sdf
bounds.head(5) # what does the result mean? why there are so many records?

what does the result mean? why there are so many records? Because each row in borough_sdf stands for a LSOA zone. when you apply the function bounds. It actually returns bounding box for each individual LSOA zone.

**In such case, how to get the minx, miny, maxx, maxy of the entire borough?**

In [ ]:
figsize=(7,13) # Why do you think I changed the figure size this way?

base = the_borough.plot(color='white', edgecolor='black')
listings.plot(ax=base, marker='o', color='red', markersize=5);

# You should get a 'crop' of London using bounding box
#get the minx, miny, maxx, maxy of the entire borough
plt.xlim([bounds.minx.min()-5, bounds.maxx.max()+5])
plt.ylim([bounds.miny.min()-5, bounds.maxy.max()+5]);

plt.title("Quantile Classification of AirBnB Sample")

print("Done.")

## Final Task: Spatial Joins of Raster file and Shapefile

As aforementioned in Lecture, raster file and shapefile can be converted in either ways. 

The task we are going to do here is to do crop air quality data in one borough - how about Kensington again? And then do simple statistics to looking at the PM 2.5 conditions in the borough.

In [ ]:
# so we have the borough
the_borough.plot()

If we do spatial join using the_borough which contains many LSOA zones, there is a chance that any points or cells fall on the boundary of LSOA might be double counted by adjacent zones. So to avoid the bias, we will first union all LSOA zones into one big area. 

Geopandas has a full set of functions to get bounding box, buffered zones, outlines, read [here](http://geopandas.org/geometric_manipulations.html) to get more information. 
**Go through the page and find which function is the right one to apply**

In [ ]:
#so you should union all LSOA zones to get the boundary of the entire zone. check 
the_boundary = the_borough.unary_union
the_boundary

'the_boundary' is just a polygon object, we have to convert it to GeoSeries, then save to shapefile.

In [ ]:
outline = gpd.GeoSeries(the_boundary)
outline.plot()

In [ ]:
outline.???(os.path.join('output', 'boundary.shp'))

Remember that we have converted raster data into pandas dataframe. see below. If you treat the (x,y) as coordinates in 2D, you actucally, converted the raster data into a vector data set, with each grid represented by a point.

In [ ]:
import georasters as gr
import os

In [ ]:
#import the airqality file just downloaded from your local folder
rasterpath = os.path.join('data','2013_Met2013_LAEI2013_PM25.asc')

# Load data
rasterdata = gr.from_file(rasterpath)

In [ ]:
raster_df = rasterdata.to_pandas()
raster_df.head(3)

Just to give you an illustration, we plot out part of the data. (The full data sets contains 5856428 points. Plotting the whole data sets will run for quite a long time)

In [ ]:
raster_df_sel = raster_df.loc[(raster_df.row <200) & (raster_df.col > 950)]
raster_df_sel.plot.scatter(x='x', y='y', c = 'value')

Ok, now we are going to get all points within the borough boundary. 

To improve the efficiency, we will do a two step process - 

- step 1 select points in bounding box - a very simple polygon composed by only four line segments

- step 2 spatial join the selected points with borough boundary - comparatively complicated polygon


**Think about why we do it in two steps!**

In [ ]:
outline = gpd.read_file(os.path.join('output', 'boundary.shp'))
outline_bounds = outline.geometry.bounds # you get the bounding box of ken_sdf

In [ ]:
outline_bounds

So easy, we even do not have to do spatial join. Just select by values. 

In [ ]:
raster_df_sel = raster_df.loc[((raster_df.x >= 522964.5807) & (raster_df.x <= 528549.6785))&((raster_df.y >= 176948.6617)&(raster_df.y <=182764.7440))]

In [ ]:
#check if the number of points has been greatly reduced.
print('the numebr of points in raster_df is %d' %len(raster_df))
print('the numebr of points in raster_df_sel is %d' %len(raster_df_sel))

Ok, to perform a spatial join in second step, we have to generate spatial data from raster_df.x and raster_df.y. You have done that before in Practial 2.

In [ ]:
raster_df_sel['geometry'] = raster_df_sel.apply(lambda loc: Point((float(loc.x), float(loc.y))), axis=1)
raster_df_sel = gpd.GeoDataFrame(raster_df_sel, geometry='???')
raster_df_sel.crs= "init: epsg:27700"

In [ ]:
outline.crs = "init: epsg:27700"

In [ ]:
print(raster_df_sel.crs)

In [ ]:
outline.plot()

In [ ]:
raster_df_sel.plot()

In [ ]:
raster_df_sel.head(5)

In [ ]:
#perform the spatial join - select all points fall in the boundary of the borough
pm25_in_borough = gpd.sjoin(raster_df_sel, outline, how="inner", op='within')

In [ ]:
len(pm25_in_borough)

In [ ]:
pm25_in_borough.head(5)

In [ ]:
pm25_in_borough.plot()

In [ ]:
#finally, what the average PM25 in this borough?



## Further explore: Raster data in GDAL




Just for your interests, though georasters provides well-wrapped and easy to use functions. GDAL is more powerful, and gives you the flexiblity to manipulate data. 

So, we will use gdal package to load raster file. With GDAL, you can read and write several different raster formats in Python. Python automatically registers all known GDAL drivers for reading supported formats when the importing the GDAL module. 
Most common file formats include for example TIFF and GeoTIFF, ASCII Grid and Erdas Imagine .img -files. read more [here](https://automating-gis-processes.github.io/2016/Lesson7-read-raster.html).

In [ ]:
from osgeo import gdal

filepath  = os.path.join('data', 'pm25_tiff.tif')

# Open the file:
airquality = gdal.Open(filepath)

# Check type of the variable 'raster'
type(airquality)

In [ ]:
print('The properties of the file:')
print('The number of rows in the raster file is %d' %airquality.RasterXSize)
print('The number of columns in the raster file is %d' %airquality.RasterYSize)
print('The number of bands of the raster file is %d' %airquality.RasterCount)
print('The projection of the raster file is %s' %airquality.GetProjection())

We mentioned in the Lecture that **raster data model allows NULL value**. The '[No Data Value]' are actually null value in the raster data. 

In [ ]:
rasterArray = airquality.ReadAsArray()

In [ ]:
rasterArray

In [ ]:
band = airquality.GetRasterBand(1)
# Get nodata value from the GDAL band object
nodata = band.GetNoDataValue()

#Create a masked array for making calculations without nodata values
rasterArray = np.ma.masked_equal(rasterArray, nodata)
type(rasterArray)

# Check again array statistics, the minimum value of rasterArray
rasterArray.???()

In [ ]:
rasterArray

In [ ]:
# Get bounding box
width = airquality.RasterXSize    
height = airquality.RasterYSize

In [ ]:
## Get bounding box
width = airquality.RasterXSize    
height = airquality.RasterYSize
gt = airquality.GetGeoTransform()
minx = gt[0]   
miny = gt[3] + width * gt[4] + height * gt[5]    
maxx = gt[0] + width * gt[1] + height * gt[2]    
maxy = gt[3]

## Get elevation
band = airquality.GetRasterBand(1)

## Set NaNs 
pmdata = band.ReadAsArray()
noDataValue = band.GetNoDataValue()
pmdata[pmdata == noDataValue] = np.nan

## Plot it using matplotlib.pyplot
plt.imshow(pmdata, cmap = 'gist_earth', extent = [minx, maxx, miny, maxy])